In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
        
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%run helpers.py

### Executive Summary

In [2]:
#this is the search parameters that i will be searching in the website
#and i can keep extending list if i want to or specify it 

search_parameters=['informatics']#informatics doesn't have that many jobs so i used it as a test subject 

# the search parameters i used in my actual scraping:
# ['data', 'analysis', 'research','informatics','statistic','business intelligence']


#converting the search parameters to use in the url, meaing i will be placing the search terms in the url itself:
def replace(lists):
    for index, item in enumerate(lists):
        lists[index]=(item.replace(' ','-'))
    return lists


In [3]:
replace(search_parameters)

# converted list
search_parameters

['informatics']

In [ ]:
# Now for the main course:
# here i will be searching all the search parameters and getting the jobs for each search parameter



# creating a dataframe and its index 
# and a temporary dictioonary
data=None
index=1 
temp_article={}

# looping through the search parameters and placing in in the url to get the jobs for each search parameter
for item in search_parameters:
    
    print item #this will give me a count and progress info of the scraping
    
    url='https://www.seek.com.au/%s-jobs?page='%item #placing in url
    
    #calculating how many pages to go through by getting the job number shown(search result number) on the site and dividing it up by 20
    # because there are 20 jobs per page
    #however the limit is 500m pages on the website so if the search result number shown has more jobs than 500 pages 
    #it will only look for 500 pages and jobs on only 500 pages 
    
    #getting the url for the search parameter
    r = requests.get(url)
    #turning it into a html doc
    html0=html.document_fromstring(r.text)
    #locating the 'job number'(serach result number) listed for the search parameter using xpath
    search_page = html0.xpath('//*[@id="SearchSummary"]/strong')
    #looping through the search_page list:
    for job_number in search_page:
        
        #converting the number to a usable int and rounding it up with .9 and 
        # +1 to make include the number in the range 
        num=int(round(float(job_number.text.replace(',',''))/20.0+.9))+1
        if num>500:
            num = 500
        else:
            num=num

    print num  #this will give me a count and progress info of the scraping
    
    #looping thourgh each page
    for page in range(1,num):
        
        print page #this will give me a count and progress info of the scraping
        
        
        # make request for that page
        r = requests.get(url+str(page))

        # turn into a BeautifulSoup object
        soup = BeautifulSoup(r.text, 'lxml')


        # find job articles on page
        articles = soup.find_all(name='article')


        # iterating through each job on the main search page:
        for article in articles:
            
            #lookng for some feature that are not in the jobs page itself
            #desrciption,job_id and level

            #description feature:
            dvg=article.find_all(name='span', attrs={'class':"bl7UwXp"})
            for z in dvg:
                temp_article['description']=z.text

            #getting the job_id and level which an attribute of the html article code:
            temp_article['job_id']=article.attrs['data-job-id']
            temp_article['level']=article.attrs['data-automation']


            #finding the url for each job so that i can go to each jobs own page
            #and grab features straight from there
            dvg=article.find_all(name='a',attrs={'class':"_1EkZJQ7"})
            for z in dvg:
                
                #finding the url for the job's own page
                s=z.attrs['href']
                #requesting html for the job's page
                r = requests.get('https://www.seek.com.au'+s)
                #convering it to html file
                html1=html.document_fromstring(r.text)
                
                #finding the remaining attributes on the page itself using xpath and temporarily adding it
                #to the 'temp_article' dictionary


                #title
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[1]/div/div/section/div/div/span/span/h1')
                for z in dvg:
                    temp_article['title']=z.text



                #company       
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[1]/div/div/section/div/div/h2/span[2]/span')
                for z in dvg:
                    temp_article['company']=z.text


                #date
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/dd[1]/span/span')
                for z in dvg:
                    temp_article['date']=z.text

                #location
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/dd[2]/span/span/strong')
                for z in dvg:
                    temp_article['location'] = z.text


                #salary
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/div[1]/dd/span/span')
                for z in dvg:
                    temp_article['salary']=z.text

                #area
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/dd[2]/span/span/span/text()')
                for z in dvg:
                    temp_article['area']=z

                #type
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/dd[3]/span/span')
                for z in dvg:
                    temp_article['type']=z.text


                #class
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/div/dd/span/span/strong')
                for z in dvg:
                    temp_article['class']=z.text

                #sub_class
                dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/div/dd/span/span/span/text()')
                for z in dvg:
                    temp_article['sub_class']=z




            #making the dataframe using the 'temp_article' dictionary, info created at the beginning of this code
            temp=pd.DataFrame(temp_article, index=[index])
            index+=1
            try:
                data=pd.concat([data,temp])

            except:
                data=temp

        
        
        


data
500
1
2
3
4
5
6
7
8
9
10
11
12


In [7]:
# saving it down as a excel file
# also i commented it out simply because i don't want it to overlap and create a new file
# print data.shape
# print data.info
# data.to_excel('./scraped.xls')

In [4]:
#reloaded the file to check it out once it was saved

data2=pd.read_excel('./scraped.xls')

In [5]:
print data2.shape
print data2.info()
data2.head()

(30312, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30312 entries, 1 to 30312
Data columns (total 12 columns):
area           30311 non-null object
class          30311 non-null object
company        21172 non-null object
date           30311 non-null object
description    30312 non-null object
job_id         30312 non-null int64
level          30312 non-null object
location       30311 non-null object
salary         9976 non-null object
sub_class      30311 non-null object
title          30311 non-null object
type           30311 non-null object
dtypes: int64(1), object(11)
memory usage: 3.0+ MB
None


,area,class,company,date,description,job_id,level,location,salary,sub_class,title,type
1,NaN,NaN,NaN,NaN,Join a talented and dynamic team in multi-chan...,35046021,premiumJob,NaN,NaN,NaN,NaN,NaN
2,"CBD, Inner West & Eastern Suburbs",Information & Communication Technology,Energy & Water Ombudsman NSW,19 Dec 2017,Seeking an experienced desktop support analyst...,35118800,premiumJob,Sydney,NaN,Business/Systems Analysts,Data Analysis and Support Officer,Part Time
3,"CBD, Inner West & Eastern Suburbs",Information & Communication Technology,NaN,21 Dec 2017,Expertise in Data Architecture frameworks e.g....,35132539,normalJob,ACT,NaN,Architects,Data Architect,Contract/Temp
4,"CBD, Inner West & Eastern Suburbs",Healthcare & Medical,NaN,21 Dec 2017,"Analyse financial, human resource, service del...",35131963,normalJob,Townsville & Northern QLD,Great remuneration & benefits!,Management,Finance and Data Manager,Contract/Temp
5,"CBD, Inner West & Eastern Suburbs",Accounting,NaN,21 Dec 2017,"Analyse financial, human resource, service del...",35131964,normalJob,Townsville & Northern QLD,Great remuneration & benefits!,Management,Finance and Data Manager,Contract/Temp


In [ ]:
#This is where is was trying alot of the stuff

In [ ]:

# make request for that page
r = requests.get('https://www.seek.com.au/informatics-jobs?page='+str(1))

# turn into a BeautifulSoup object
soup = BeautifulSoup(r.text, 'lxml')


# find job article on page
articles = soup.find_all(name='article')


# r = requests.get(url)
html0=html.document_fromstring(r.text)
search_page = html0.xpath('//*[@id="SearchSummary"]/strong')
for job_number in search_page:
    
    num=int(round(float(job_number.text.replace(',',''))/20.0+.9))+1
    if num>500:
        num = 500
    else:
        num=num
    print num
    print range(1,num)
    
# iterating through each job:
for article in articles:
    
    #description feature:
    dvg=article.find_all(name='span', attrs={'class':"bl7UwXp"})
    for z in dvg:
        temp_article['description']=z.text
    
    #getting the job_id and level which an attribute of the html article code:
    temp_article['job_id']=article.attrs['data-job-id']
    temp_article['level']=article.attrs['data-automation']
            
    
    #finding the url for each job so that i can to to each jobs page
    #and grab features straight from there
    dvg=article.find_all(name='a',attrs={'class':"_1EkZJQ7"})
    for z in dvg:

        s=z.attrs['href']

        r = requests.get('https://www.seek.com.au'+s)

        html1=html.document_fromstring(r.text)
        
        
        #title
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[1]/div/div/section/div/div/span/span/h1')
        for z in dvg:
            temp_article['title']=z.text
    


        #company       
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[1]/div/div/section/div/div/h2/span[2]/span')
        for z in dvg:
            temp_article['company']=z.text


        #date
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/dd[1]/span/span')
        for z in dvg:
            temp_article['date']=z.text

        #location
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/dd[2]/span/span/strong')
        for z in dvg:
            temp_article['location'] = z.text
   

        #salary
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/div[1]/dd/span/span')
        for z in dvg:
            temp_article['salary']=z.text

        #area
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/dd[2]/span/span/span/text()')
        for z in dvg:
            temp_article['area']=z
            
        #type
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/dd[3]/span/span')
        for z in dvg:
            temp_article['type']=z.text
            
            
        #class
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/aside/span/div[1]/div/section/dl/div/dd/span/span/strong')
        for z in dvg:
            temp_article['class']=z.text
            
        #sub_class
        dvg = html1.xpath('//*[@id="app"]/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div/article/div/div[1]/div/section/dl/div/dd/span/span/span/text()')
        for z in dvg:
            temp_article['sub_class']=z

    
               
     
    #making the dataframe
    temp=pd.DataFrame(temp_article, index=[index])
    index+=1
    try:
        x=pd.concat([x,temp])
        
    except:
        x=temp
        


In [ ]:
x.head(20)